In [1]:
import psycopg2
import config

## Postgres SQL Connection

In [11]:
conn = psycopg2.connect(user = config.DATABASE_CONFIG['user'],
                        password = config.DATABASE_CONFIG['password'],
                        host = config.DATABASE_CONFIG['host'],
                        port = config.DATABASE_CONFIG['port'],
                        database = config.DATABASE_CONFIG['dbname'])

conn.set_session(autocommit=True)

In [3]:
print(conn)

<connection object at 0x0000020ADC3E3450; dsn: 'user=postgres password=xxx dbname=course_data host=127.0.0.1 port=5432', closed: 0>


In [13]:
cur = conn.cursor()

## Basic Queries

In [7]:
query = "SELECT * FROM student_enrollment"

cur.execute(query)
cur.fetchall()

[(1, 'CS110'),
 (1, 'CS180'),
 (1, 'CS210'),
 (2, 'CS107'),
 (2, 'CS220'),
 (3, 'CS110'),
 (3, 'CS180'),
 (4, 'CS220'),
 (5, 'CS110'),
 (5, 'CS180'),
 (5, 'CS210'),
 (5, 'CS220'),
 (6, 'CS110'),
 (7, 'CS110'),
 (7, 'CS210')]

## Advance Queries

Functions and Conditions

In [15]:
query = "SELECT last_name || ' ' || 'works in the '|| department || ' department' FROM professors"

cur.execute(query)
cur.fetchall()

[('Chong works in the Science department',),
 ('Brown works in the Math department',),
 ('Jones works in the History department',),
 ('Wilson works in the Astronomy department',),
 ('Miller works in the Agriculture department',),
 ('Williams works in the Law department',)]

In [16]:
query = "SELECT 'It is ' || (salary > 95000) || ' that professor ' || last_name || ' is highly paid' FROM professors"

cur.execute(query)
cur.fetchall()

[('It is false that professor Chong is highly paid',),
 ('It is true that professor Brown is highly paid',),
 ('It is false that professor Jones is highly paid',),
 ('It is true that professor Wilson is highly paid',),
 ('It is false that professor Miller is highly paid',),
 ('It is true that professor Williams is highly paid',)]

In [18]:
query = "SELECT MAX(salary) as higest_salary, MIN(salary) as lowest_salary FROM professors WHERE last_name != 'Wilson'"

cur.execute(query)
cur.fetchall()

[(105000, 67000)]

In [19]:
query = "SELECT MIN(hire_date) FROM professors"

cur.execute(query)
cur.fetchall()

[(datetime.date(2001, 6, 5),)]

In [17]:
query = "SELECT last_name, UPPER(SUBSTRING(department, 1, 3)) as department, salary, hire_date FROM professors"

cur.execute(query)
cur.fetchall()

[('Chong', 'SCI', 88000, datetime.date(2006, 4, 18)),
 ('Brown', 'MAT', 97000, datetime.date(2002, 8, 22)),
 ('Jones', 'HIS', 67000, datetime.date(2009, 11, 17)),
 ('Wilson', 'AST', 110000, datetime.date(2005, 1, 15)),
 ('Miller', 'AGR', 82000, datetime.date(2008, 5, 8)),
 ('Williams', 'LAW', 105000, datetime.date(2001, 6, 5))]

In [20]:
query = "SELECT state FROM fruit_imports GROUP BY state ORDER BY SUM(supply) desc LIMIT 1"

cur.execute(query)
cur.fetchall()

[('Texas',)]

In [21]:
query = "SELECT season, MAX(cost_per_unit) highest_cost_per_unit FROM fruit_imports GROUP BY season"

cur.execute(query)
cur.fetchall()

[('Winter', Decimal('0.22')),
 ('Summer', Decimal('0.68')),
 ('All Year', Decimal('0.75')),
 ('Spring', Decimal('0.60')),
 ('Fall', Decimal('0.35'))]

In [22]:
query = "SELECT state FROM fruit_imports GROUP BY state, name HAVING COUNT(name) > 1"

cur.execute(query)
cur.fetchall()

[('Texas',)]

In [23]:
query = "SELECT season, COUNT(name) FROM fruit_imports GROUP BY season HAVING count(name) = 3 OR count(name) = 4"

cur.execute(query)
cur.fetchall()

[('Summer', 4), ('All Year', 3), ('Spring', 4)]

In [24]:
query = "SELECT state, SUM(supply * cost_per_unit) total_cost FROM fruit_imports GROUP BY state ORDER BY total_cost desc LIMIT 1"

cur.execute(query)
cur.fetchall()

[('Illinois', Decimal('25375.00'))]

In [27]:
query = "SELECT student_name FROM students WHERE student_no IN (SELECT student_no FROM student_enrollment WHERE course_no IN ( SELECT course_no  FROM courses WHERE course_title IN ('Physics', 'US History')))"

cur.execute(query)
cur.fetchall()

[('Michael',), ('Doug',), ('Samantha',), ('Pete',), ('Ralph',)]

In [28]:
query = "SELECT * FROM students WHERE age = (SELECT MAX(age) FROM students)"

cur.execute(query)
cur.fetchall()

[(6, 'Arnold', 22)]

In [29]:
query = "SELECT student_name FROM students WHERE student_no IN (SELECT student_no FROM (SELECT student_no, COUNT(course_no) course_cnt FROM STUDENT_ENROLLMENT GROUP BY student_no ORDER BY course_cnt desc LIMIT 1)a)"

cur.execute(query)
cur.fetchall()

[('Ralph',)]

Write a query that displays 3 columns. The query should display the fruit and it's total supply along with a category of either LOW, ENOUGH or FULL. Low category means that the total supply of the fruit is less than 20,000. The enough category means that the total supply is between 20,000 and 50,000. If the total supply is greater than 50,000 then that fruit falls in the full category.

In [8]:
query = "SELECT name, total_supply,                                                     \
        CASE WHEN total_supply < 20000 THEN 'LOW'                                       \
            WHEN total_supply >= 20000 AND total_supply <= 50000 THEN 'ENOUGH'          \
            WHEN total_supply > 50000 THEN 'FULL'                                       \
        END as category                                                                 \
        FROM (                                                                          \
                SELECT name, sum(supply) total_supply                                   \
                FROM fruit_imports                                                      \
                GROUP BY name                                                           \
            ) a"

cur.execute(query)
cur.fetchall()

[('Cherry', 62150, 'FULL'),
 ('Orange', 35000, 'ENOUGH'),
 ('Coconut', 15200, 'LOW'),
 ('Lime', 40400, 'ENOUGH'),
 ('Cantaloupe', 8000, 'LOW'),
 ('Apricot', 69500, 'FULL'),
 ('Mango', 30650, 'ENOUGH'),
 ('Grape', 72500, 'FULL'),
 ('Pear', 67750, 'FULL'),
 ('Avocado', 27000, 'ENOUGH'),
 ('Apple', 32900, 'ENOUGH')]

Taking into consideration the supply column and the cost_per_unit column, you should be able to tabulate the total cost to import fruits by each season. 

In [14]:
query = "SELECT SUM(CASE WHEN season = 'Winter' THEN total_cost end) as Winter_total,   \
            SUM(CASE WHEN season = 'Summer' THEN total_cost end) as Summer_total,       \
            SUM(CASE WHEN season = 'Spring' THEN total_cost end) as Spring_total,       \
            SUM(CASE WHEN season = 'Fall' THEN total_cost end) as Spring_total,         \
            SUM(CASE WHEN season = 'All Year' THEN total_cost end) as Spring_total      \
        FROM (                                                                          \
        select season, sum(supply * cost_per_unit) total_cost                           \
        from fruit_imports                                                              \
        group by season) a"

cur.execute(query)
cur.fetchall()

[(Decimal('10072.50'),
  Decimal('19623.00'),
  Decimal('29930.00'),
  Decimal('29035.00'),
  Decimal('22688.00'))]

In [30]:
query = "SELECT student_name, course_no, min(last_name) FROM ( SELECT student_name, se.course_no, p.last_name FROM students s  \
        INNER JOIN student_enrollment se ON s.student_no = se.student_no \
        INNER JOIN teach t              \
        ON se.course_no = t.course_no INNER JOIN professors p         \
        ON t.last_name = p.last_name) a \
        GROUP BY student_name, course_no    \
        ORDER BY student_name, course_no;"

cur.execute(query)
cur.fetchall()

[('Arnold', 'CS110', 'Brown'),
 ('Doug', 'CS107', 'Williams'),
 ('Doug', 'CS220', 'Jones'),
 ('Michael', 'CS110', 'Brown'),
 ('Michael', 'CS180', 'Brown'),
 ('Michael', 'CS210', 'Jones'),
 ('Pete', 'CS220', 'Jones'),
 ('Ralph', 'CS110', 'Brown'),
 ('Ralph', 'CS180', 'Brown'),
 ('Ralph', 'CS210', 'Jones'),
 ('Ralph', 'CS220', 'Jones'),
 ('Samantha', 'CS110', 'Brown'),
 ('Samantha', 'CS180', 'Brown')]

ROLLUP in SQL aggregation is used to perform multiple levels of analysis with a single query. ROLLUP performs aggregations across multiple dimensions at different levels within a single SQL query

In [35]:
query = "SELECT continent, country, city, sum(units_sold) FROM sales \
        GROUP BY ROLLUP(continent, country, city)"

cur.execute(query)
cur.fetchall()

[(None, None, None, 68000),
 ('Asia', 'Japan', 'Tokyo', 5000),
 ('North America', 'Canada', 'Montreal', 5000),
 ('Europe', 'France', 'Paris', 5000),
 ('Europe', 'UK', 'London', 6000),
 ('North America', 'Canada', 'Toronto', 10000),
 ('Asia', 'China', 'Shanghai', 3000),
 ('Asia', 'China', 'Hong Kong', 7000),
 ('Europe', 'UK', 'Manchester', 12000),
 ('North America', 'Canada', 'Vancouver', 15000),
 ('Asia', 'Japan', None, 5000),
 ('Asia', 'China', None, 10000),
 ('Europe', 'UK', None, 18000),
 ('Europe', 'France', None, 5000),
 ('North America', 'Canada', None, 30000),
 ('Asia', None, None, 15000),
 ('North America', None, None, 30000),
 ('Europe', None, None, 23000)]

SQL CUBE is a data structure, more or less like a matrix or a two-dimensional array that makes it possible to aggregate values and summary reports on multiple axes and provides a more detailed analysis by performing grouping of data along with more than one column